## The journeys of Bizantyne iconographic models

Import main libraries and retrieve the geographical coordinates starting from the GeoNames ID

In [1]:
import pandas as pd
import geocoder

matthiae = pd.read_csv("matthiae.csv")

for idx, row in matthiae.iterrows():
    geonameid = str(row["authority_orginalloc"]).replace("geonames:","")
    g = geocoder.geonames(geonameid, method='details', key='manuveggi99')
    matthiae.at[idx, "lat"] = g.lat
    matthiae.at[idx, "lng"] = g.lng

In [2]:
def popup_html(row):

    if len(str(row["author"])) > 3:
        author = str(row["author"])
    else:
        author = ""

    description = str(row["description"])

    if str(row["start_date"]) == str(row["end_date"]):
        date = str(row["start_date"])
    else:
        date = "{0}-{1}".format(str(row["start_date"]),str(row["end_date"]))

    image = str(row["path"])

    comment = str(row["comment"])
    
    
    html = """
        <!DOCTYPE html>
        <html>
        <h5>""" + description + """</h5>
        <p>""" + author + """</p>
        <p>""" + date + """</p>
        <center><img src=\"""" + image + """\" alt="reproduction" width=50%></center>
        <p>""" + comment + """</p>
        </html>
        """
    return html

In [6]:
import folium
from folium import IFrame

def showmap(myDf):

    map_obj = folium.Map(location = [43, 20], zoom_start = 4)

    group1 = folium.FeatureGroup(name='OPERE. Esordi (VI - IX sec)')
    group2 = folium.FeatureGroup(name='OPERE. La dinastia comnena (1057 - 1185)')
    group3 = folium.FeatureGroup(name='OPERE. La dinastia paleologa (1259 - 1453)')

    map_obj.add_child(group1)
    map_obj.add_child(group2)
    map_obj.add_child(group3)

    #Add mercator projection for different years
    merc  = "mercproj/1185_Z6.png"
    merc1 = "mercproj/1205_Z6.png"
    merc2 = "mercproj/1262_Z6.png"

    img = folium.raster_layers.ImageOverlay(
        name="SCENARIO. Contesto geopolitico del 1185",
        image=merc,
        bounds=[[-84.48, -186.4], [97, 173.6]],
        opacity=0.6, 
        interactive=True,
        cross_origin=False,
        zindex=1,
        #show=False
    )

    img1 = folium.raster_layers.ImageOverlay(
        name="SCENARIO. Contesto geopolitico del 1204",
        image=merc1,
        bounds=[[-84.48, -186.4], [97, 173.6]],
        opacity=0.6,
        interactive=True,
        cross_origin=False,
        zindex=1,
        show=False
    )

    img2 = folium.raster_layers.ImageOverlay(
        name="SCENARIO. Contesto geopolitico del 1261",
        image=merc2,
        bounds=[[-84.48, -186.4], [97, 173.6]],        
        opacity=0.6,
        interactive=True,
        cross_origin=False,
        zindex=1,
        show=False
    )

    img.add_to(map_obj)
    img1.add_to(map_obj)
    img2.add_to(map_obj)

    for idx, row in myDf.iterrows():
        

        # Define marker class
        if row["period"] == "prodromi":
            currentGroup = group1
            currentColor = "darkpurple"
        elif row["period"] == "comneno":
            currentGroup = group2
            currentColor = "red"
        elif row["period"] == "paleologo":
            currentGroup = group3
            currentColor = "orange"

        html = popup_html(row)
        popup = folium.Popup(folium.Html(html, script=True), max_width=500, max_height=300)
        
        folium.Marker(
            [row["lat"], row["lng"]],
            popup= popup,
            icon=folium.Icon(color=currentColor, icon="map-marker", prefix='fa') 
        ).add_to(currentGroup)
    
    # add layer control to map (allows layers to be turned on or off)
    folium.LayerControl(collapsed=False).add_to(map_obj)
    
    return map_obj

showmap(matthiae).save("firstmap.html")
#showmap(matthiae)